In [ ]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git@duot5

In [2]:
import pyterrier as pt
pt.init()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
monoT5 = MonoT5ReRanker()
duoT5 = DuoT5ReRanker()

In [4]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./index_path"
indexer = pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter())

[INFO] [starting] building docstore
[INFO] If you have a local copy of http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/23e5607081191b153738e81fbd834680
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter: 0it [00:00, ?it/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.8%| 16.4k/2.13M [00:00<00:16, 127kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:11, 183kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 3.9%| 81.9k/2.13M [00:00<00:10, 201kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 6.9%| 147k/2.13M [00:00<00:07, 270kB/s] 
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 14.6%| 311k/2.13M [00:00<00:03, 

In [5]:
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25") % 100
mono_pipeline = bm25 >> pt.text.get_text(dataset, "text") >> monoT5
duo_pipeline = mono_pipeline % 10 >> duoT5

In [6]:
pt.Experiment(
  [
   bm25,
   mono_pipeline,
   duo_pipeline,
  ],
  dataset.get_topics(),
  dataset.get_qrels(),
  names=["BM25", "BM25 >> monoT5", "BM25 >> monoT5 >> duoT5"],
  eval_metrics=["map", "recip_rank", "P.10", "ndcg_cut.10"]
)

[INFO] NumExpr defaulting to 2 threads.


,name,map,recip_rank,P_10,ndcg_cut_10
0,BM25,0.272523,0.725587,0.352688,0.446609
1,BM25 >> monoT5,0.295071,0.750839,0.413978,0.490703
2,BM25 >> monoT5 >> duoT5,0.157311,0.655615,0.413978,0.464646
